In [35]:
#get some packages
import numpy as np
import pandas as pd
import os
import dask.dataframe as dd
from tqdm import tqdm

In [143]:
#get Opioid NDCs from CDC MME dataset and set NDC field to str to maintain lead zeros.
#source: https://www.cdc.gov/drugoverdose/resources/data.html
NDC = pd.read_csv("OpioidNDCs_SAS.csv", dtype={'NDC':'str', 'NDC_Numeric':'float'}) 

In [144]:
#peek at NDC file datatypes
NDC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14549 entries, 0 to 14548
Data columns (total 13 columns):
NDC                      14549 non-null object
NDC_Numeric              14549 non-null float64
PRODNME                  14549 non-null object
GENNME                   14549 non-null object
Master_Form              14549 non-null object
Class                    14549 non-null object
Drug                     14549 non-null object
LongShortActing          14549 non-null object
DEAClassCode             14549 non-null int64
Strength_Per_Unit        14545 non-null float64
UOM                      14544 non-null object
MME_Conversion_Factor    14324 non-null float64
NDC_No_Lead_Zeros        14549 non-null int64
dtypes: float64(3), int64(2), object(8)
memory usage: 1.4+ MB


In [38]:
#peek at the first few rows of the NDC file
NDC.head(3)

,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor,NDC_No_Lead_Zeros
0,2035102,2035102.0,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50.0,MG,0.23,2035102
1,2035103,2035103.0,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50.0,MG,0.23,2035103
2,2035133,2035133.0,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50.0,MG,0.23,2035133


In [117]:
#peek at Open Payments Data
#file_path = 'open_payments_2017g.csv'
file_path = 'open_payments_2013g.csv'
df_tmp = pd.read_csv(file_path, nrows=5)
df_tmp.head()

,Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Physician_Profile_ID,Physician_First_Name,Physician_Middle_Name,Physician_Last_Name,Physician_Name_Suffix,...,NDC_of_Associated_Covered_Drug_or_Biological3,NDC_of_Associated_Covered_Drug_or_Biological4,NDC_of_Associated_Covered_Drug_or_Biological5,Name_of_Associated_Covered_Device_or_Medical_Supply1,Name_of_Associated_Covered_Device_or_Medical_Supply2,Name_of_Associated_Covered_Device_or_Medical_Supply3,Name_of_Associated_Covered_Device_or_Medical_Supply4,Name_of_Associated_Covered_Device_or_Medical_Supply5,Program_Year,Payment_Publication_Date
0,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,329424,Daniel,L,McLachlan,NaN,...,NaN,NaN,NaN,TearLab Osmolarity System,NaN,NaN,NaN,NaN,2013,01/18/2019
1,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,45253,Sandeep,K,Bhatia,NaN,...,NaN,NaN,NaN,TearLab Osmolarity System,NaN,NaN,NaN,NaN,2013,01/18/2019
2,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,22706,PAULA,M,FRACASSO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01/18/2019
3,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,316315,DAVID,M,HOLTZMAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01/18/2019
4,UNCHANGED,Covered Recipient Physician,NaN,NaN,NaN,168313,LINDA,A,MORROW,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,01/18/2019


In [118]:
#peek at datatypes
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 65 columns):
Change_Type                                                          5 non-null object
Covered_Recipient_Type                                               5 non-null object
Teaching_Hospital_CCN                                                0 non-null float64
Teaching_Hospital_ID                                                 0 non-null float64
Teaching_Hospital_Name                                               0 non-null float64
Physician_Profile_ID                                                 5 non-null int64
Physician_First_Name                                                 5 non-null object
Physician_Middle_Name                                                5 non-null object
Physician_Last_Name                                                  5 non-null object
Physician_Name_Suffix                                                0 non-null float64
Recipient_Primary_Business

In [119]:
#set column datatypes and limit to only required columns (2013)
coltypes = {'Change_Type': 'str',
              'Covered_Recipient_Type': 'str', 
              'Physician_Profile_ID': 'str',
              'Physician_First_Name': 'str',
              'Physician_Middle_Name':'str',
              'Physician_Last_Name':'str',
              'Physician_Name_Suffix':'str',
              'Recipient_Primary_Business_Street_Address_Line1':'str',
              'Recipient_Primary_Business_Street_Address_Line2': 'str',
              'Recipient_City':'str',
              'Recipient_State': 'str',
              'Recipient_Zip_Code': 'str',
              'Recipient_Country':'str',
              'Physician_Primary_Type': 'str',
              'Physician_Specialty': 'str',
              'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID': 'str',
              'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name': 'str',
              'Total_Amount_of_Payment_USDollars': 'str',
              'Date_of_Payment': 'str',
              'Physician_Ownership_Indicator': 'str',
              'Product_Indicator': 'str',
              'Name_of_Associated_Covered_Drug_or_Biological1': 'str',
              'NDC_of_Associated_Covered_Drug_or_Biological1':'str',
              'Name_of_Associated_Covered_Drug_or_Biological2': 'str',
              'NDC_of_Associated_Covered_Drug_or_Biological2':'str',
              'Name_of_Associated_Covered_Drug_or_Biological3': 'str',
              'NDC_of_Associated_Covered_Drug_or_Biological3':'str',
              'Name_of_Associated_Covered_Drug_or_Biological4': 'str',
              'NDC_of_Associated_Covered_Drug_or_Biological4':'str'}
cols = list(coltypes.keys())

In [120]:
#The Open Payments Data is very big so we need to put it in chunks to bring it in for subsetting
chunksize = 1_000_000 

In [146]:
%%time
df_us_doc_subset = [] # list to hold the batch dataframe (2013)
df_ndc_subset = []


for df_chunk in tqdm(pd.read_csv(file_path, usecols=cols, dtype=coltypes, chunksize=chunksize)):
     
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
    df_chunk['Date_of_Payment'] = df_chunk['Date_of_Payment'].str.slice(0, 16)
    df_chunk['Date_of_Payment'] = pd.to_datetime(df_chunk['Date_of_Payment'], utc=True, format='%m/%d/%Y')
    
    #clean the data up a bit and add some additional fields
    df_chunk['File_Year'] = '2013'
    df_chunk['File_Type'] = 'General'
    df_chunk['zip'] = df_chunk['Recipient_Zip_Code'].astype(str)
    df_chunk.zip = df_chunk.zip.str[:5]
    df_chunk['key'] = df_chunk['Physician_Last_Name']+'-'+df_chunk['Physician_First_Name']+'-'+df_chunk['Recipient_State']+'-'+df_chunk['zip']
    
    #do some subsetting
    filtered_us_doc = df_chunk[(df_chunk['Recipient_Country'] == 'United States') & (df_chunk['Covered_Recipient_Type']=='Covered Recipient Physician')]
    
    # Append the chunk to list and merge all
    df_us_doc_subset.append(filtered_us_doc)
    
    #Do some more data management specific to NDC
    df_chunk['NDC'] = df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].str.replace('-', '')
    df_chunk1 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('1')]
    df_chunk2 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('2')]
    df_chunk3 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('3')]
    df_chunk4 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('4')]
    df_chunk5 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('5')]
    df_chunk6 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('6')]
    df_chunk7 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('7')]
    df_chunk8 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('8')]
    df_chunk9 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('9')]
    df_chunk0 = df_chunk[df_chunk['NDC_of_Associated_Covered_Drug_or_Biological1'].astype(str).str.startswith('0')]
    frames = [df_chunk1, df_chunk2, df_chunk3, df_chunk4, df_chunk5, df_chunk6, df_chunk7, df_chunk8, df_chunk9, df_chunk0]
    df_chunk = pd.concat(frames)
    df_chunk['NDC_Numeric'] = df_chunk['NDC'].astype(float)
    
    #do some subsetting based on NDC
    filtered_ndc = pd.merge(df_chunk, NDC, on='NDC_Numeric', how='inner')
    
    # Append the chunk to list and merge all
    df_ndc_subset.append(filtered_ndc)
   



























0it [00:00, ?it/s]

























1it [00:22, 22.98s/it]

























2it [00:45, 22.89s/it]

























3it [01:08, 22.93s/it]

























4it [01:34, 23.86s/it]

























5it [01:39, 18.27s/it]

Wall time: 1min 40s


In [147]:
# Merge all dataframes into one dataframe
df_us_doc_subset_13 = pd.concat(df_us_doc_subset)

# Delete the dataframe list to release memory
del df_us_doc_subset

# See what we have loaded
df_us_doc_subset_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4145413 entries, 0 to 4171053
Data columns (total 33 columns):
Change_Type                                                      object
Covered_Recipient_Type                                           object
Physician_Profile_ID                                             object
Physician_First_Name                                             object
Physician_Middle_Name                                            object
Physician_Last_Name                                              object
Physician_Name_Suffix                                            object
Recipient_Primary_Business_Street_Address_Line1                  object
Recipient_Primary_Business_Street_Address_Line2                  object
Recipient_City                                                   object
Recipient_State                                                  object
Recipient_Zip_Code                                               object
Recipient_Country      

In [148]:
# Merge all dataframes into one dataframe
df_ndc_subset_13 = pd.concat(df_ndc_subset)

# Delete the dataframe list to release memory
del df_ndc_subset

# See what we have loaded
df_ndc_subset_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 0
Data columns (total 47 columns):
Change_Type                                                      17 non-null object
Covered_Recipient_Type                                           17 non-null object
Physician_Profile_ID                                             17 non-null object
Physician_First_Name                                             17 non-null object
Physician_Middle_Name                                            15 non-null object
Physician_Last_Name                                              17 non-null object
Physician_Name_Suffix                                            1 non-null object
Recipient_Primary_Business_Street_Address_Line1                  17 non-null object
Recipient_Primary_Business_Street_Address_Line2                  3 non-null object
Recipient_City                                                   17 non-null object
Recipient_State                                             

In [73]:
#freqency for qualitative var
df_cat_subset_13['NDC_Numeric'].value_counts()

5045857810     73270
0169643990     70351
0456080060     69166
5045814001     65333
0456120130     60872
0169406090     59058
0005970135     51665
0001860370     51187
0005970075     49424
0001860777     46250
6340230430     45584
5914800613     43277
0074433902     42232
0003100751     37053
6362933571     36559
5551371001     34178
6559770118     33438
0002751001     32375
0169633990     31508
6678021904     31034
0071101268     30788
0456114030     30482
7598701003     29891
0051846233     29884
0003089321     28282
0003089431     27598
6476462530     24517
0003421541     24289
0597014030     23723
0078050115     23499
               ...  
4353818345         1
5254400442         1
0078032705         1
4970222248         1
6340270104         1
7648900102         1
7604510320         1
0052031510         1
6731339022         1
6872710003         1
1886012101         1
78034061           1
6872710002         1
0024079375         1
0003029328         1
0009518101         1
4284710330   

In [60]:
#descriptive stats for quantitative var
df_cat_subset_13['Total_Amount_of_Payment_USDollars'].describe()

count     4145413
unique      71907
top        100.00
freq        16396
Name: Total_Amount_of_Payment_USDollars, dtype: object

In [ ]:
#peek at Open Payments Data
file_path = 'open_payments_2017g.csv'
df_tmp = pd.read_csv(file_path, nrows=5)
df_tmp.head()

In [15]:
#set column datatypes and limit to only required columns (2017)
coltypes = {'Change_Type': 'str',
              'Covered_Recipient_Type': 'str', 
              'Physician_Profile_ID': 'str',
              'Physician_First_Name': 'str',
              'Physician_Middle_Name':'str',
              'Physician_Last_Name':'str',
              'Physician_Name_Suffix':'str',
              'Recipient_Primary_Business_Street_Address_Line1':'str',
              'Recipient_Primary_Business_Street_Address_Line2': 'str',
              'Recipient_City':'str',
              'Recipient_State': 'str',
              'Recipient_Zip_Code': 'str',
              'Recipient_Country':'str',
              'Physician_Primary_Type': 'str',
              'Physician_Specialty': 'str',
              'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID': 'str',
              'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name': 'str',
              'Total_Amount_of_Payment_USDollars': 'float',
              'Date_of_Payment': 'str',
              'Physician_Ownership_Indicator': 'str',
              'Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1': 'str',
              'Product_Category_or_Therapeutic_Area_1': 'str',
              'Name_of_Associated_Covered_Drug_or_Biological1': 'str',
              'Associated_Drug_or_Biological_NDC_1':'str'}
cols = list(coltypes.keys())

In [54]:
%%time
df_cat_subset = [] # list to hold the batch dataframe (2017)
df_ndc_merge = [] # list to hold the batch dataframe (2017)

for df_chunk in tqdm(pd.read_csv(file_path, usecols=cols, dtype=coltypes, chunksize=chunksize)):
     
    # Neat trick from https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
    # Using parse_dates would be much slower!
    df_chunk['Date_of_Payment'] = df_chunk['Date_of_Payment'].str.slice(0, 16)
    df_chunk['Date_of_Payment'] = pd.to_datetime(df_chunk['Date_of_Payment'], utc=True, format='%m/%d/%Y')
    
    #clean the data up a bit and add some additional fields
    df_chunk['NDC'] = df_chunk['Associated_Drug_or_Biological_NDC_1'].str.replace('-', '')
    df_chunk['NDC_Numeric'] = df_chunk['NDC'].astype(float)
    df_chunk['File_Year'] = '2017'
    df_chunk['File_Type'] = 'General'
    df_chunk['zip'] = df_chunk['Recipient_Zip_Code'].astype(str)
    df_chunk.zip = df_chunk.zip.str[:5]
    df_chunk['key'] = df_chunk['Physician_Last_Name']+'-'+df_chunk['Physician_First_Name']+'-'+df_chunk['Recipient_State']+'-'+df_chunk['zip']
    
  #do some subsetting
    filtered_cat = df_chunk[(df_chunk['Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1'] == 'Drug') & (df_chunk['Recipient_Country'] == 'United States') & (df_chunk['Covered_Recipient_Type']=='Covered Recipient Physician')]
    filtered_ndc = pd.merge(filtered_cat, NDC, on='NDC_Numeric', how='inner')
    
    # Append the chunk to list and merge all
    df_cat_subset.append(filtered_cat)
    df_ndc_merge.append(filtered_ndc) 



0it [00:00, ?it/s]

KeyError: 'Associated_Drug_or_Biological_NDC_1'

In [ ]:
df_cat_subset_final.info()

In [26]:
df_cat_subset_13.to_csv(r'C:\Users\santo\desktop\df_cat_subset_13.csv')

In [21]:
manufacturers = df_cat_subset_17[c(["Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name"].value_counts(), 
manufacturers.to_csv(r'C:\Users\santo\desktop\manufacturer_freq.csv')

In [34]:
import sys
fil=sys.argv[1]
csvfilename = open(fil, 'r').readlines()
file = 1
for j in range(len(csvfilename)):
    if j % 1000000 == 0:
        open(str(fil)+str(fil)+'.csv', 'w+').writelines(csvfilename[j:j+1000000])
        file +=1

FileNotFoundError: [Errno 2] No such file or directory: '-f'